# Titanic - Czyszczenie danych i przygotowanie do analizy
Mateusz Lindel  
29.12.2020

## Cel sprawozdania
Celem sprawozdania jest przygotowanie zbioru do analizy. W zbiorze titanica należy znaleźć błędy, a następnie oczyścić dane z błedów, które mogą wpłynąć na dogłębniejszą analizę.

## Przygtowanie
Na potrzeby czyszczenia danych wykorzystano biblioteki numpy, math, pandas oraz scikt-learn (sklearn).

In [1]:
# zaimportowanie potrzebnych bibliotek
import math
import numpy as np
import pandas as pd
import locale
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

'en_US.UTF-8'

Kolejnym krokiem jest wczytanie pliku oraz próba wyświetlenia kilku pierwszych wierszy. Następnie wyświetlono dodatkowe informacje o zbiorze - jego wymiarowości oraz informacje o kolumnach.

In [2]:
titanic = pd.read_csv('TitanicMess.tsv', sep='\t', index_col='PassengerId') #separator \t, kolumna indeksowana 'PassengerId'
titanic.head(10) 


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,"8,4583",NaN,Q,Titanic
7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,"51,8625",E46,S,Titanic
8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,"21,075",NaN,S,Titanic
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,"11,1333",NaN,S,Titanic


In [3]:
titanic.shape # informacja o wymiarowości - 892 wierszy po 12 kolumn każdy

(892, 12)

In [4]:
titanic.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'ship'],
      dtype='object')

In [5]:
titanic.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 892 entries, 1 to 1000
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Survived  892 non-null    int64 
 1   Pclass    892 non-null    int64 
 2   Name      892 non-null    object
 3   Sex       892 non-null    object
 4   Age       719 non-null    object
 5   SibSp     892 non-null    int64 
 6   Parch     892 non-null    int64 
 7   Ticket    892 non-null    object
 8   Fare      892 non-null    object
 9   Cabin     207 non-null    object
 10  Embarked  890 non-null    object
 11  ship      892 non-null    object
dtypes: int64(4), object(8)
memory usage: 62.7+ KB


In [6]:
titanic.describe()
#wyświetlenie dodatkowych informacji z danych w kolumnach 
#(ilość, średnia, odchylenie standardowe, minimalna wartość, maksymalna wartość itp)

,Survived,Pclass,SibSp,Parch
count,892.000000,892.000000,892.000000,892.000000
mean,0.386771,2.307175,0.525785,0.371076
std,0.487284,0.836750,1.102190,0.768468
min,0.000000,1.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,0.000000
50%,0.000000,3.000000,0.000000,0.000000
75%,1.000000,3.000000,1.000000,0.000000
max,1.000000,3.000000,8.000000,5.000000


## Analiza i czyszczenie zbioru

Na początek wyświetlone zostaną informacje ile oraz w jakich kolumnach występują puste wartości. Dodatkowo wyświetlona zostanie informacja o typach kolumn, które zostały automatycznie ustawione podczas wczytywania zbioru.


In [7]:
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         173
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

In [8]:
titanic.dtypes

Survived     int64
Pclass       int64
Name        object
Sex         object
Age         object
SibSp        int64
Parch        int64
Ticket      object
Fare        object
Cabin       object
Embarked    object
ship        object
dtype: object

Jak można zauważyć kolumny **Age** oraz **Fare** są typem object, a wydaje się być słusznie założenie, że oczekiwano typu liczbowego. Kolumna **Age**, **Cabin** oraz **Embarked** posiadają również braki. W pierwszej kolejności zajmę się kolumną Age - należy ustawić jej odpowiedni typ.

In [9]:
wrongConvert = 0
for index, row in titanic.iterrows():
    try:
        int(row['Age'])
    except ValueError:
        wrongConvert += 1
        print(f'Błąd konwertowania wieku dla ID {index}')
        print(row['Age'])
        
print(f'Ilość nieskonwertowanych wierszy {wrongConvert}')

Błąd konwertowania wieku dla ID 6
nan
Błąd konwertowania wieku dla ID 18
nan
Błąd konwertowania wieku dla ID 20
nan
Błąd konwertowania wieku dla ID 27
nan
Błąd konwertowania wieku dla ID 29
nan
Błąd konwertowania wieku dla ID 30
nan
Błąd konwertowania wieku dla ID 32
.9
Błąd konwertowania wieku dla ID 33
nan
Błąd konwertowania wieku dla ID 37
nan
Błąd konwertowania wieku dla ID 43
nan
Błąd konwertowania wieku dla ID 46
nan
Błąd konwertowania wieku dla ID 47
nan
Błąd konwertowania wieku dla ID 48
nan
Błąd konwertowania wieku dla ID 49
nan
Błąd konwertowania wieku dla ID 56
nan
Błąd konwertowania wieku dla ID 58
28,5
Błąd konwertowania wieku dla ID 65
nan
Błąd konwertowania wieku dla ID 66
nan
Błąd konwertowania wieku dla ID 77
nan
Błąd konwertowania wieku dla ID 78
nan
Błąd konwertowania wieku dla ID 79
0,83
Błąd konwertowania wieku dla ID 83
nan
Błąd konwertowania wieku dla ID 88
nan
Błąd konwertowania wieku dla ID 96
nan
Błąd konwertowania wieku dla ID 102
nan
Błąd konwertowania wieku

Jak widać problematyczne dla konwertowania wieku są wartości puste (nan) oraz wartości posiadające wiek z przecinkiem 
bądź kropką.
Zadecydowano aby wykorzystać typ float, zamieniając również przecinki na kropki by konwersja do typu float przebiegła pomyślnie.

In [10]:
"""
Pomocnicza funkcja dla konwersji wartości na typ float z kolumny dla wiersza
"""
def ConvertToFloat(x, columnName):
    return float(str(x[columnName]).replace(",", "."))

wrongConvert = 0
for index, row in titanic.iterrows():
    try:
        ConvertToFloat(row, "Age")
    except ValueError:
        wrongConvert += 1
        print(f'Błąd konwertowania wieku dla ID {index}')
        print(row['Age'])
        
print(f'Ilość nieskonwertowanych wierszy {wrongConvert}')

Ilość nieskonwertowanych wierszy 0


Tym razem brak nieskonwertowanych wartości - dodatkową informacją jest fakt, że typ float poprawnie poradził sobie z brakiem wartości i nie został odebrany jako błędny. Możemy zatem przejść do oczyszczania danych dla kolumny Age - zastąpienia wartości typem float oraz zamiana typu kolumny z object na float.
Pod uwagę wzięto również fakt, że wiek musi być większy 0. Jeżeli wiek jest liczbą ujemną zostanie on zastąpiony wartością NaN.

In [11]:
for index, row in titanic.iterrows():
    try:
        age = ConvertToFloat(row, "Age")
        if(age < 0):
            titanic.at[index, "Age"] = np.NaN
        else:
            titanic.at[index, "Age"] = age
    except ValueError:
        titanic.at[index, "Age"] = np.NaN

for index, row in titanic.iterrows():
    print(row["Age"])

22.0
38.0
26.0
35.0
35.0
nan
54.0
2.0
27.0
14.0
4.0
58.0
20.0
4.0
14.0
55.0
2.0
nan
31.0
nan
35.0
34.0
15.0
4.0
8.0
38.0
nan
19.0
nan
nan
40.0
0.9
nan
66.0
28.0
42.0
nan
21.0
18.0
14.0
40.0
27.0
nan
3.0
19.0
nan
nan
nan
nan
18.0
7.0
21.0
49.0
29.0
65.0
nan
21.0
28.5
5.0
11.0
22.0
38.0
45.0
4.0
nan
nan
29.0
19.0
17.0
26.0
32.0
16.0
21.0
26.0
32.0
25.0
nan
nan
0.83
30.0
22.0
29.0
nan
28.0
17.0
33.0
16.0
nan
23.0
24.0
29.0
20.0
46.0
26.0
59.0
nan
71.0
23.0
34.0
34.0
28.0
nan
21.0
33.0
37.0
28.0
21.0
nan
38.0
nan
47.0
14.5
22.0
20.0
17.0
21.0
70.5
29.0
24.0
2.0
21.0
nan
32.5
32.5
54.0
12.0
nan
24.0
nan
45.0
33.0
20.0
47.0
29.0
25.0
23.0
19.0
37.0
16.0
24.0
nan
22.0
24.0
19.0
18.0
19.0
27.0
9.0
36.5
42.0
51.0
22.0
55.5
40.5
nan
51.0
16.0
30.0
nan
nan
44.0
40.0
26.0
17.0
1.0
9.0
nan
45.0
nan
28.0
61.0
4.0
1.0
21.0
56.0
18.0
nan
50.0
30.0
36.0
nan
nan
9.0
1.0
4.0
nan
nan
45.0
40.0
36.0
32.0
19.0
19.0
3.0
44.0
58.0
nan
42.0
nan
24.0
28.0
nan
34.0
45.5
18.0
2.0
32.0
26.0
16.0
40.0
24.0
35.0
22.

In [12]:
titanic[["Age"]] = titanic[["Age"]].astype(float).round(1) # zamiana typu kolumny do float
titanic.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare         object
Cabin        object
Embarked     object
ship         object
dtype: object

Konwersja kolumny Age w zbiorze danych została zakończona sukcesem. Można zatem zająć się pustymi wartościami dla tej kolumny.

Jedną z metod uzupełniania brakujących danych jest podstawienie średniej wartości dla wierszy w występują braki. Zdecydowano się oczyścić zbiór przy wykorzystaniu tej metody.

In [13]:
titanic['Age'] = titanic['Age'].replace(np.NaN, titanic['Age'].mean())
titanic.head(30)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,"7,25",NaN,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,"71,2833",C85,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,"53,1",C123,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,"8,05",NaN,S,Titanic
6,0,3,"Moran, Mr. James",male,36.050349,0,0,330877,"8,4583",NaN,Q,Titanic
7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,"51,8625",E46,S,Titanic
8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,"21,075",NaN,S,Titanic
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,"11,1333",NaN,S,Titanic


Po tych działań w zbiorze danych kolumna **Age** została oczyszczona i przygotowana do dalszej analizy.

Kolumna **Fare** identycznie jak kolumna **Age** jest typem object. Zastosowano zatem identyczną metodę do podmiany tych wartości. Dla ujemnych wartości również podstawiono wartość NaN.

In [14]:
for index, row in titanic.iterrows():
    try:
        fare = ConvertToFloat(row, "Fare")
        if(fare < 0):
            titanic.at[index, "Fare"] = np.NaN
        else:
            titanic.at[index, "Fare"] = fare
    except ValueError:
        titanic.at[index, "Fare"] = np.NaN

In [15]:
titanic['Fare'] = titanic['Fare'].replace(np.NaN, titanic['Fare'].mean())
titanic.dtypes


Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
ship         object
dtype: object

In [16]:
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64

Pozbyto się zatem pustych wartości z kolumny **Fare** oraz zamieniona typ kolumny na float.

Kolumna **Emabarked**, posiadająca informacje na temat portu z którego dana osoba wsiadła, posiada 2 wiersza z pustymi wartościami. Umieszczono zatem losowo C - nie wpłynie to w żaden sposób na dalszą analizę danych.

In [17]:
titanic['Embarked'] = titanic['Embarked'].replace(np.NaN, 'C')
titanic.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age           0
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      0
ship          0
dtype: int64

Kolejną kolumną z pustymi wartościami jest **Cabin**. Niestety, braków jest bardzo dużo, dokładnie 685, co daje nam ponad 75% braków w całym zbiorze danych. Zdecydowano o usunięciu kolumny - danych jest na tyle mało, że nie ma możliwości jej analizy, a ponadto jakakolwiek próba uzupełnienia ich wprowadzi tylko błędne wyniki podczas analizy.

In [18]:
titanic.drop('Cabin', axis = 1, inplace = True)
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ship
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,Titanic
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,Titanic
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,Titanic
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,Titanic
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,Titanic


Dane zostały oczyszczone z pustych wartości. Kolejnym etapem będzie zatem badanie, czy kolumny posiadają poprawne wartości np. czy w kolumnie **Sex** istnieją tylko wartości male bądź female.

Dla porządku sprawdźmy wszystkie kolumny. Na początek sprawdźmy kolumnę **Survived** - oczekujemy, że wystąpią w niej dwie wartości 0 oraz 1.

In [19]:
print(titanic['Survived'].unique())

[0 1]


Jak widać kolumna zawiera oczekiwane wartości i nie trzeba wykonywać żadnych działań.

W identyczny sposób zostaną sprawdzono kolejne kolumny
- **Pclass** - oczekiwane wartości: 3, 2, bądź 1
- **Sex** - oczekiwane wartości: male, female
- **Embarked** - oczekiwane wartości: S, C, Q

In [20]:
print('Unikalne wartości dla kolumny Pclass ', titanic['Pclass'].unique())
print('Unikalne wartości dla kolumny Sex ', titanic['Sex'].unique())
print('Unikalne wartości dla kolumny Embarked ', titanic['Embarked'].unique())

Unikalne wartości dla kolumny Pclass  [3 1 2]
Unikalne wartości dla kolumny Sex  ['male' 'female' 'malef' 'mal' 'fem' 'femmale']
Unikalne wartości dla kolumny Embarked  ['S' 'C' 'Q' 'So' 'Co' 'Qe']


Jak widać, dla kolumny **Sex** istnieje więcej wartości niż można było oczekiwać, nie wydaje się żeby te płcie odnosiły się do LGBT, zatem zostaną one podmienione na male oraz female.
Kolumna **Embarked** również posiada zbyt wiele wartości. Wartości po prostu powtarzają się tylko są inaczej zapisane, nastąpi ich korekta.

In [21]:
titanic['Sex'] = titanic['Sex'].replace({
    'malef': 'male',
    'mal': 'male',
    'fem': 'female',
    'femmale': 'female'
}) # wykorzystanie typu słownika do podmiany wartości

titanic['Embarked'] = titanic['Embarked'].replace({
    'So': 'S',
    'Co': 'C',
    'Qe': 'Q',
}) 

print('Unikalne wartości dla kolumny Sex ', titanic['Sex'].unique())
print('Unikalne wartości dla kolumny Embarked ', titanic['Embarked'].unique())

Unikalne wartości dla kolumny Sex  ['male' 'female']
Unikalne wartości dla kolumny Embarked  ['S' 'C' 'Q']


Kolumna **Sex** oraz **Emabarked** zawierają już oczekiwane wartości.

Kolejnym krokiem będzie sprawdzenie danych pod kątem duplikatów.

In [22]:
titanic[titanic.duplicated(keep=False)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ship
PassengerId,,,,,,,,,,,
11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4.0,1,1,PP 9549,16.7,S,Titanic
11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4.0,1,1,PP 9549,16.7,S,Titanic
11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4.0,1,1,PP 9549,16.7,S,Titanic
225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38.0,1,0,19943,90.0,S,Titanic
225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38.0,1,0,19943,90.0,S,Titanic


Jak wynika z działania - kilka wierszy powatarza się. Należy usunąć niepotrzebne duplikaty.

In [23]:
titanic.drop_duplicates(keep = False, inplace = True)
titanic[titanic.duplicated(keep=False)]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ship
PassengerId,,,,,,,,,,,


Z zbioru danych pozbyto się oczywistych duplikatów. Sprawdźmy jeszcze, czy istnieją duplikaty pod kątem kolumny **Name**

In [24]:
titanic[titanic.duplicated(subset=['Name'])]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,ship
PassengerId,,,,,,,,,,,


Jak widać, żadne nazwiska nie powtarzają się w zbiorze danych. Zatem ten etap czyszczenia danych można uznać za zakończony.

Na koniec zostawiona została jeszcze kolumna **ship**, której wartości prezentują się następująco.

In [25]:
print('Unikalne wartości dla kolumny Ship ', titanic['ship'].unique())

Unikalne wartości dla kolumny Ship  ['Titanic']


Jak widać w tej kolumnie istnieje tylko wartość **Titanic**, a z wcześniejszych analiz nie ma informacji o brakach danych, zatem w tym przypadku można spokojnie tą kolumnę usunąć, ponieważ wartości są zupełnie zbędne w dalszych analizach tego zbioru.

In [26]:
titanic.drop('ship', axis = 1, inplace = True)

Po wykonaniu wszystkich kroków zbiór danych titanic jest gotowy do dalszych prac a etap oczysczania danych uznano za zakończony.

Oto kilka informacji o zbiorze po wykonaniu czyszczenia danych.

In [27]:
titanic.head(10)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,S
6,0,3,"Moran, Mr. James",male,36.050349,0,0,330877,8.4583,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,17463,51.8625,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,347742,11.1333,S


In [28]:
titanic.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,887.000000,887.000000,887.000000,887.000000,887.000000,887.000000
mean,0.383315,2.307779,36.154353,0.523112,0.369786,32.151785
std,0.486468,0.835834,148.642125,1.104718,0.769561,49.754536
min,0.000000,1.000000,0.400000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,22.000000,0.000000,0.000000,7.895800
50%,0.000000,3.000000,32.500000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,36.050349,1.000000,0.000000,31.000000
max,1.000000,3.000000,4435.000000,8.000000,5.000000,512.329200


In [29]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 887 entries, 1 to 1000
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  887 non-null    int64  
 1   Pclass    887 non-null    int64  
 2   Name      887 non-null    object 
 3   Sex       887 non-null    object 
 4   Age       887 non-null    float64
 5   SibSp     887 non-null    int64  
 6   Parch     887 non-null    int64  
 7   Ticket    887 non-null    object 
 8   Fare      887 non-null    float64
 9   Embarked  887 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 62.4+ KB


Ostatnim krokiem jest wyeksportowanie oczyszczonych danych do pliku tsv

In [30]:
titanic.to_csv("TitanicCleaned.tsv", sep = "\t")